In [1]:
%pip install -qU langgraph langchain-openai langchain-core langchain-community langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 10.4 MB/s eta 0:00:00


In [2]:
import os
with open("/content/api_key.txt") as archivo:
  apikey = archivo.read()
  os.environ["OPENAI_API_KEY"] =apikey

In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [4]:
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
### Carga

loader =  WebBaseLoader(web_paths=("https://es.wikipedia.org/wiki/Cuzco",))
docs = loader.load()
### fragmentacion
text_expliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
all_split = text_expliter.split_documents(docs)
#### almacenamiento
vectore_store = InMemoryVectorStore(embedding)
_ = vectore_store.add_documents(all_split)


In [13]:
retriever_chain = vectore_store.as_retriever()

In [14]:
query= "¿Cuando llegó al poder Pachacútec?"
respuesta = retriever_chain.invoke(query)
respuesta

[Document(id='18de49f0-f9f0-44f5-ae40-fdf9a14d211d', metadata={'source': 'https://es.wikipedia.org/wiki/Cuzco', 'title': 'Cuzco - Wikipedia, la enciclopedia libre', 'language': 'es'}, page_content='Representación de las cuatro divisiones del Imperio incaico o Tahuantinsuyu, que partían del Cuzco, la ciudad capital con forma de puma.\nSe atribuye al gobernante Pachacútec el haber hecho del Cuzco un centro espiritual y político. Pachacútec llegó al poder en 1438, y él y su hijo Túpac Yupanqui dedicaron cinco décadas a la organización y conciliación de los diferentes grupos tribales bajo su dominio, entre ellos los lupacas y los collas. Durante el periodo de Pachacútec y Túpac Yupanqui, el dominio de Cuzco llegó hasta Quito, al norte, y hasta el río Maule, al sur, integrando a su cultura a los habitantes de 4500\xa0km de cadenas montañosas.\nTambién se cree que el diseño original de la ciudad es obra de Pachacútec. El plano del Cuzco antiguo tiene forma de puma delineado, con la plaza cen

In [16]:
tool_cuzco = retriever_chain.as_tool(
    name ="datos_cuzco",
    description="Herramienta para obtener infomacion de la ciudad de cuzco"
)

In [19]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

toolkit=[tool_cuzco]

model = ChatOpenAI(model="gpt-4.1-2025-04-14")
agent = create_react_agent(model, toolkit)

# Invocamos al agente
config = {"configurable": {"thread_id": "000"}}
response = agent.invoke({"messages": [HumanMessage(content="cuando Francisco Pizarro refundó a la usanza española la ciudad del Cuzco")]}, config=config)
response['messages'][-1].pretty_print()

================================== Ai Message ==================================

Francisco Pizarro refundó a la usanza española la ciudad del Cuzco el 23 de marzo de 1534. En esa ocasión se estableció como plaza de Armas el lugar que todavía ocupa en la ciudad moderna, que también era la plaza principal durante el incanato, rodeada por los palacios de los antiguos soberanos incas. Al día siguiente, el 24 de marzo de 1534, se fundó el cabildo de la ciudad y se nombraron los primeros alcaldes y regidores.


In [20]:
response

{'messages': [HumanMessage(content='cuando Francisco Pizarro refundó a la usanza española la ciudad del Cuzco', additional_kwargs={}, response_metadata={}, id='9b077407-c8e3-4a7c-a024-4d02b22c94fb'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Rz0OOQhvXqtdlsPBK4WflJ5e', 'function': {'arguments': '{"__arg1":"refundación española"}', 'name': 'datos_cuzco'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 79, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_beec22d258', 'id': 'chatcmpl-BZBF3tsDxj52835ddiyERx4bqJFgk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--75603884-f5b3-4084-a3d0-739669c66b72-0', tool_calls=[{'nam